# Parsing and Cleaning PHEME RNR Dataset Events

This notebook performs data-cleaning and aggrigation into thread-level data. It also provides a useful sanity check. Run all the cells in this notebook to generate thread-level CSV files in the `data/threads` directory.

In [1]:
# Load dependencies for this Jupyter Notebook
import pandas as pd
import numpy as np
import networkx as nx
from functools import reduce
from lib.util import fetch_tweets

## Parsing and Cleaning Data
This step takes the raw PHEME rumor dataset and saves it tabular format as CSV file. The original PHEME dataset consists of JSON files organized into directories by event and category (rumor or non-rumor). These three functions below parse the data, save it as a CSV file (if necessary), and load it into this notebook as a Pandas DataFrame from the "cached" CSV file.

In [2]:
dataset_name="sydneysiege"
gw = fetch_tweets(dataset_name)

##  Tweet Level Features

| Name/Column       | Description                   | Type   | Notes  |
|-------------------|-------------------------------|--------| ------ |
| is_rumor          | Was this classified as rumor  | "bool" (`int`) | *Classification done by journalists* |
| thread            | Source tweet id               | `str`  |                                                   |
| in_reply_tweet    | Tweet ID in reply to          | `str`  |                                                   |
| event             | Name of the PHEME event       | `str`  | Corresponds to event in the PHEME dataset         |
| tweet_id          | Unique ID for tweet           | `str`  | This field is the ID referenced in `in_reply_tweet`     |
| is_source_tweet   | Was this classified as rumor  | "bool" (`int`) |                                                   |
| in_reply_user     | User ID in reply to           | `str`  |                                                   |
| user_id           | Twitter User's ID             | `str`  | This field is the ID referenced in `in_reply_user` |
| tweet_length      | Number of characters in tweet | `int`  |                                                   |
| urls_count        | Number of URLS in tweet       | `int`  |                                                   |
| hashtags_count    | Number of hashtags in tweet   | `int`  |                                                   |
| retweet_count     | Times the tweet was retweeted | `int`  |                                                   |
| favorite_count    | Number of times favorited     | `int`  |                                                   |
| mentions_count    | Number of users mentioned     | `int`  |                                                   |
| is_truncated      | Is this tweet truncated       | "bool" (`int`) | Did User type > 140 characters. [See Tweet updates](https://developer.twitter.com/en/docs/tweets/tweet-updates) |
| created              | Datetime Tweet was created    | `datetime` | |
| has_smile_emoji      | Does Tweet contain ""?        | "bool" (`int`) | 😊 is the smile emoji |
| user.tweets_count    | User's tweet total, currently | `int`  | |
| user.verified        | Is Twitter user verified?     | "bool" (`int`) |                                                   |
| user.followers_count | Total number of followers  | `int` | |
| user.listed_count    | ?? | `int` | | 
| user.friends_count   | ?? | `int` | |
| user.time_zone       | Timezone of the user's Twitter account | `str` | |
| user.desc_length     | Length of the user's biographic description | `int` |
| user.has_bg_img      | Does user have a profile background image?  | "bool" (`int`) |
| user.default_pric    | Does the user have the default profile picture | "bool" (`int`) |
| user.created_at      | Date and time Twitter account was activated | `datetime` | |
| user.utc_dist        | TK | `int` | See [this blog post time and the Twitter API](https://zacharyst.com/2017/04/05/assigning-the-correct-time-to-a-twee) |

## Germanwings Crash

In [3]:
gw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23996 entries, 0 to 23995
Data columns (total 59 columns):
is_rumor                23996 non-null int64
thread                  23996 non-null object
in_reply_tweet          23996 non-null object
event                   23996 non-null object
tweet_id                23996 non-null object
is_source_tweet         23996 non-null int64
in_reply_user           23996 non-null object
user_id                 23996 non-null object
tweet_length            23996 non-null int64
symbol_count            23996 non-null int64
user_mentions           23996 non-null int64
urls_count              23996 non-null int64
media_count             23996 non-null int64
hashtags_count          23996 non-null int64
retweet_count           23996 non-null int64
favorite_count          23996 non-null int64
mentions_count          23996 non-null int64
is_truncated            23996 non-null int64
created                 23996 non-null float64
has_smile_emoji         2399

The `.head` method prints out the 5 first rows in the dataframe

In [4]:
gw.head()

,is_rumor,thread,in_reply_tweet,event,tweet_id,is_source_tweet,in_reply_user,user_id,tweet_length,symbol_count,...,sentimentscore,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,has_url_in_text
0,0,544427042616254465,nan,sydneysiege,544427042616254465,1,nan,61436584,139,0,...,0.166667,5,6,0,4,2,1,0,0,0
1,0,544427042616254465,5.4442704261625446e+17,sydneysiege,545322709756805120,0,61436584.0,887450286,133,0,...,0.000000,5,5,0,3,2,0,1,0,0
2,0,544427042616254465,5.445083586532475e+17,sydneysiege,544508565705072640,0,19317766.0,17872080,48,0,...,0.600000,3,2,0,0,1,0,0,0,0
3,0,544427042616254465,5.4442704261625446e+17,sydneysiege,544505594561167361,0,61436584.0,30726225,48,0,...,0.500000,3,0,1,0,0,0,0,0,0
4,0,544427042616254465,5.4442704261625446e+17,sydneysiege,544506491684659200,0,61436584.0,19317766,38,0,...,0.000000,3,1,0,0,0,0,0,0,0


## Boolean Columns

The `describe` method will give summary information about each column in the dataframe. Each of these columns, except `is_truncated` should have two unique values.

Just for a sanity check. The cell below converts these boolean columns into value of type `bool` and describes them.

In [5]:
bool_columns = ["is_rumor", "is_source_tweet", "is_truncated", 
                "has_smile_emoji", "user.verified", "user.has_bg_img", 
                "user.default_pic", "sensitive", "has_place", "has_coords", "user.notifications"]

gw[bool_columns].astype(bool).describe(include="bool")

,is_rumor,is_source_tweet,is_truncated,has_smile_emoji,user.verified,user.has_bg_img,user.default_pic,sensitive,has_place,has_coords,user.notifications
count,23996,23996,23996,23996,23996,23996,23996,23996,23996,23996,23996
unique,2,2,1,2,2,2,2,2,2,2,1
top,False,False,False,False,False,True,False,False,False,False,False
freq,15320,22775,23996,23973,22943,21776,14425,23933,23039,23481,23996


Some columns in some PHEME events have only one unique value for all tweets. Instead of dropping them, we'll just be aware of them because they may vary across PHEME datasets.

In [6]:
for col in gw.columns:
    if len(gw[col].unique()) == 1:
        print("Warning, column `%s` only has one unique value \"%s\"" % (col, gw[col][0]))

Warning, column `event` only has one unique value "sydneysiege"
Warning, column `symbol_count` only has one unique value "0"
Warning, column `is_truncated` only has one unique value "0"
Warning, column `user.notifications` only has one unique value "0"
Warning, column `Adverb` only has one unique value "0"


The tweet-level data, we'll use in our data analysis will be in this form.

In [7]:
gw.describe()

,is_rumor,is_source_tweet,tweet_length,symbol_count,user_mentions,urls_count,media_count,hashtags_count,retweet_count,favorite_count,...,sentimentscore,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,has_url_in_text
count,23996.000000,23996.000000,23996.000000,23996.0,23996.000000,23996.000000,23996.000000,23996.000000,23996.000000,23996.000000,...,23996.000000,23996.000000,23996.000000,23996.000000,23996.000000,23996.000000,23996.000000,23996.000000,23996.0,23996.000000
mean,0.361560,0.050883,92.492207,0.0,1.662319,0.100558,0.073429,0.235873,22.363602,20.978038,...,0.027869,5.512002,2.343266,0.902942,0.645441,0.150567,0.210660,0.388648,0.0,0.136398
std,0.480462,0.219764,38.931510,0.0,0.970727,0.312172,0.260845,0.622993,697.656358,1080.436569,...,0.311927,2.882990,1.782173,1.004158,0.932062,0.427781,0.526763,0.703738,0.0,0.343218
min,0.000000,0.000000,3.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,59.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,98.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,5.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,1.000000,0.000000,130.000000,0.0,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.136364,7.000000,4.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.000000
max,1.000000,1.000000,152.000000,0.0,11.000000,3.000000,1.000000,12.000000,99524.000000,149783.000000,...,1.000000,25.000000,11.000000,7.000000,9.000000,5.000000,7.000000,6.000000,0.0,1.000000


## Thread Level Features

* **Bold features** represent high performing features identified in C. Buntain and J. Golbeck, ["Automatically Identifying Fake News in Popular Twitter Threads"](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8118443&isnumber=8118402)
* Features that are normalized are normalized by thread length


| Name                | Description                               | Type    | Notes |
| ---                 | ---                                       | ---     | ----- |
| thread              | Tweet ID of the source tweet              | `str`   | |
| favorite_count      | Normalized favorite total                 | `float` | |
| retweet_count       | Normlaized retweet total                  | `float` | |
| **hashtags_count**  | Normlaized hashtag total                  | `float` | |
| urls_count          | URL total normalized by thread length     | `float`  | |
| user.tweets_count   | Total tweets by thread users              | `float` | |
| event               | Name of PHEME event                       | `str`  | |
| is_rumor            | Either rumor or nonrumor                  | `bool` | |
| thread_length       | Number of tweets in the thread            | `int`  | |
| user.has_bg_img     | Ratio of users who have bg image          | `float`| |
| user.default_pic    | Ratio of users with default profile pic   | `float`| |
| **has_smile_emoji** | Number of smile emojis in the thread      | `int`  | 😊 is the smile emoji |
| user.verified       | Count of verified users in the thread normalized by thread length     | `float`  | |
| **src.followers_count** | The number of followers of the original poster of the thread. | `int` | |
| src.listed_count    | TODO | `int` | |
| src.user_verified   | TODO | `int` | |
| src.tweets_total    | TODO | `int` | |
| reply_var           | The variance in the timestamps of responses to the source tweet | `float` |
| src_age             | Difference in src user's creation and tweet creation            | `int`   | Measured in seconds |
| time_to_first_resp  | The difference between tweet creation datetime and 1st reply    | `int`   | Measured in seconds |
| time_to_last_resp   | The difference between tweet creation datetime and last reply   | `int`   | Measured in seconds |

In [8]:
def agg_tweets_by_thread(df):
    
    shared = lambda x: 1 - len(set(x)) / len(x)
    shared.__name__ = "shared"

    funcs = [np.mean, sum, np.var]
    agg_props = {
        "favorite_count": funcs,
        "user_mentions": funcs,
        "media_count": funcs,
        "sensitive": funcs,
        "has_place": funcs,
        "has_coords": funcs,
        "retweet_count": funcs,
        "hashtags_count": funcs + [shared],
        "urls_count": funcs,
        "user.tweets_count": funcs,
        "is_rumor": max,
        "tweet_id": len,
        "user.has_bg_img": funcs,
        "has_quest": funcs,
        "has_exclaim": funcs,
        "has_quest_or_exclaim": funcs,
        "user.default_pic": funcs,
        "has_smile_emoji": funcs,
        "user.verified": funcs,
        "user.name_length": funcs,
        "user.handle_length": funcs,
        "user.profile_sbcolor": funcs,
        "user.profile_bgcolor": funcs,
        
        "hasperiod": funcs,
        "number_punct": funcs,
        "negativewordcount" : funcs,
        "positivewordcount" : funcs,
        "capitalratio" : funcs,
        "contentlength" : funcs,
        "sentimentscore" : funcs,
        "Noun" : funcs,
        "Verb" : funcs,
        "Adjective" : funcs,
        "Pronoun" : funcs,
        "Adverb": funcs,
    }
    rename = {
        "tweet_id": "thread_length",
        "has_url":"url_proportion",
    }

    def g(x):
        # Add size of largest user-to-user conversation component in each thread        
        d = []
        thread_tweets = list(x["tweet_id"])
        G = nx.from_pandas_edgelist(df[df.tweet_id.isin(thread_tweets)], "user_id", "in_reply_user")
        Gc = max(nx.connected_component_subgraphs(G), key=len)
        d.append(nx.number_connected_components(G))
        d.append(nx.diameter(Gc))
        return pd.Series(d, index=["component_count", "largest_cc_diameter"])
    
    # Step 0: Build graph-based features
    graph = df.groupby("thread").apply(g)
    
    # Step 1: Build simple aggregate features
    agg = df.groupby("thread")\
        .agg(agg_props)\
        .rename(columns=rename)
    
    agg.columns = [ "_".join(x) for x in agg.columns.ravel() ]
    agg = agg.rename(columns={"is_rumor_max": "is_rumor", "thread_length_len": "thread_length"})
    
    # Step 2: Builds some features off the source tweet, which has tweet_id == thread            
    src = df[df["is_source_tweet"] == 1][["thread",
                                          "user.followers_count", 
                                          "user.listed_count",
                                          "user.verified",
                                          "created",
                                          "user.created_at",
                                          "user.tweets_count"]] \
                         .rename(columns={"user.followers_count": "src.followers_count",
                                          "user.listed_count": "src.listed_count",
                                          "user.verified": "src.user_verified",
                                          "user.created_at": "src.created_at",
                                          "user.tweets_count": "src.tweets_total"})
    
    # Step 3: Build features off of the reply tweets
    def f(x):
        d = []
        
        # Get various features from the distribution of times of reply tweet
        d.append(min(x["created"]))
        d.append(max(x["created"]))
        d.append(np.var(x["created"]))
                
        return pd.Series(d, index=["first_resp", "last_resp","resp_var"])
        
    replies = df[df["is_source_tweet"] == False] \
        .groupby("thread") \
        .apply(f)

    graph_features = df.groupby("thread").apply(g)
    
    dfs = [agg, src, replies, graph]
    thrd_data = reduce(lambda left, right: pd.merge(left,right, on="thread"), dfs)
    
    # Step 3: Add miscelaneous features
    # Remember timestamps increase as time progresses
    # src.created_at < created < first_resp < last_resp
    thrd_data["time_to_first_resp"] = thrd_data["first_resp"] - thrd_data["created"]
    thrd_data["time_to_last_resp"] = thrd_data["last_resp"] - thrd_data["created"]
    
    return thrd_data

In [9]:
gw_thrds = agg_tweets_by_thread(gw)
gw_thrds.columns.values

array(['thread', 'user.verified_mean', 'user.verified_sum',
       'user.verified_var', 'number_punct_mean', 'number_punct_sum',
       'number_punct_var', 'thread_length', 'user.has_bg_img_mean',
       'user.has_bg_img_sum', 'user.has_bg_img_var', 'urls_count_mean',
       'urls_count_sum', 'urls_count_var', 'user_mentions_mean',
       'user_mentions_sum', 'user_mentions_var', 'positivewordcount_mean',
       'positivewordcount_sum', 'positivewordcount_var',
       'user.profile_bgcolor_mean', 'user.profile_bgcolor_sum',
       'user.profile_bgcolor_var', 'has_exclaim_mean', 'has_exclaim_sum',
       'has_exclaim_var', 'Noun_mean', 'Noun_sum', 'Noun_var',
       'user.name_length_mean', 'user.name_length_sum',
       'user.name_length_var', 'media_count_mean', 'media_count_sum',
       'media_count_var', 'user.profile_sbcolor_mean',
       'user.profile_sbcolor_sum', 'user.profile_sbcolor_var',
       'hashtags_count_mean', 'hashtags_count_sum', 'hashtags_count_var',
       'hashtag

In [10]:
gw_thrds.describe()

,user.verified_mean,user.verified_sum,user.verified_var,number_punct_mean,number_punct_sum,number_punct_var,thread_length,user.has_bg_img_mean,user.has_bg_img_sum,user.has_bg_img_var,...,created,src.created_at,src.tweets_total,first_resp,last_resp,resp_var,component_count,largest_cc_diameter,time_to_first_resp,time_to_last_resp
count,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,...,1.173000e+03,1.173000e+03,1173.000000,1.173000e+03,1.173000e+03,1.173000e+03,1173.000000,1173.000000,1.173000e+03,1.173000e+03
mean,0.069777,0.885763,0.067245,5.142717,99.625746,19.255169,20.416027,0.904372,18.538789,0.082491,...,1.418628e+12,1.258588e+12,54261.992327,1.418628e+12,1.418656e+12,2.571665e+14,3.086957,3.253197,5.920571e+05,2.815097e+07
std,0.087313,1.178118,0.081256,2.100673,102.446226,19.313091,20.183395,0.111175,18.866639,0.087406,...,1.947740e+07,6.175650e+10,53508.094305,2.008682e+07,4.820095e+07,9.975376e+14,1.338945,1.731509,5.514800e+06,4.450653e+07
min,0.000000,0.000000,0.000000,0.666667,2.000000,0.000000,2.000000,0.333333,1.000000,0.000000,...,1.418598e+12,1.167702e+12,32.000000,1.418599e+12,1.418601e+12,0.000000e+00,1.000000,1.000000,0.000000e+00,4.800000e+04
25%,0.000000,0.000000,0.000000,3.769231,44.000000,7.383399,10.000000,0.854167,9.000000,0.000000,...,1.418609e+12,1.223102e+12,11383.000000,1.418609e+12,1.418625e+12,8.527076e+11,2.000000,2.000000,4.800000e+04,3.426000e+06
50%,0.050000,1.000000,0.050000,4.756757,77.000000,13.392077,18.000000,0.939394,16.000000,0.058824,...,1.418626e+12,1.244628e+12,34716.000000,1.418627e+12,1.418647e+12,7.666538e+12,3.000000,3.000000,9.600000e+04,9.717000e+06
75%,0.100000,1.000000,0.099206,6.107143,120.000000,23.194444,23.000000,1.000000,21.000000,0.135338,...,1.418645e+12,1.300516e+12,96016.000000,1.418646e+12,1.418666e+12,8.105548e+13,4.000000,4.000000,1.980000e+05,3.398500e+07
max,0.750000,27.000000,0.500000,15.900000,1646.000000,246.763636,342.000000,1.000000,325.000000,0.500000,...,1.418659e+12,1.417251e+12,512276.000000,1.418744e+12,1.418940e+12,1.259862e+16,13.000000,12.000000,1.313100e+08,3.178670e+08


In [11]:
fn = "data/threads/%s.csv" % dataset_name
gw_thrds.to_csv(fn, index=False)
"Wrote data to %s" % fn

'Wrote data to data/threads/sydneysiege.csv'